<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [44]:
import os
import numpy as np
from torchvision import datasets
from tqdm import tqdm
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torchvision.transforms as transforms

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

transform = transforms.Compose([transforms.RandomRotation(10),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ColorJitter(
                                    brightness=0.4,
                                    contrast=0.4,
                                    saturation=0.4,
                                    hue=0.1
                                ),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])

#specify a simpler transform for validation and test
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
data_dir = os.getcwd() + '\\data\\'
train_dir = os.path.join(data_dir, 'train\\')
valid_dir = os.path.join(data_dir, 'valid\\')
test_dir = os.path.join(data_dir, 'test\\')
train_data = datasets.ImageFolder(train_dir, transform=transform)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)

loaders = {'train' : train_loader, 'valid': valid_loader, 'test': test_loader}

In [46]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, valid_loss_min = np.Inf):
    """returns trained model"""    
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(loaders['train'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)            
            loss.backward()
            optimizer.step()
            
            ## record the average training loss, using something like
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(tqdm(loaders['valid'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [40]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(tqdm(loaders['test'])):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [35]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 
#model_transfer = models.vgg16(pretrained=True)
model_transfer = models.densenet161(pretrained=True)

# Freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False
  
n_inputs = model_transfer.classifier.in_features

# new layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, 3)

model_transfer.classifier = last_layer

if use_cuda:
    model_transfer = model_transfer.cuda()

In [36]:
import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
#optimizer_transfer = optim.Adagrad(model_transfer.classifier[6].parameters(), lr=0.01)
optimizer_transfer = optim.Adagrad(model_transfer.classifier.parameters(), lr=0.01)

In [37]:
# load the model that got the best validation accuracy from previous training sessions
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

# train the model
n_epochs=15
model_transfer = train(n_epochs, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')


 67%|██████████████████████████████████████████████████████▎                          | 67/100 [27:32<13:24, 24.38s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [41:16<00:00, 26.02s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:58<00:00, 20.34s/it]


Epoch: 1 	Training Loss: 0.844891 	Validation Loss: 0.811929
Validation loss decreased (inf --> 0.811929).  Saving model ...



 67%|██████████████████████████████████████████████████████▎                          | 67/100 [27:00<13:48, 25.12s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [41:03<00:00, 25.05s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [03:02<00:00, 21.44s/it]


...


 67%|██████████████████████████████████████████████████████▎                          | 67/100 [29:19<14:08, 25.72s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [42:58<00:00, 24.39s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:59<00:00, 20.81s/it]


Epoch: 3 	Training Loss: 0.649070 	Validation Loss: 0.772986
Validation loss decreased (0.811929 --> 0.772986).  Saving model ...



  9%|███████▍                                                                          | 9/100 [03:42<37:53, 24.99s/it]

KeyboardInterrupt: 

In [38]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

In [42]:
test(loaders, model_transfer, criterion_transfer, use_cuda)



  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

  3%|██▊                                                                                | 1/30 [00:24<11:54, 24.63s/it]

  7%|█████▌                                                                             | 2/30 [00:49<11:32, 24.73s/it]

 10%|████████▎                                                                          | 3/30 [01:14<11:05, 24.65s/it]

 13%|███████████                                                                        | 4/30 [01:41<11:01, 25.46s/it]

 17%|█████████████▊                                                                     | 5/30 [02:06<10:34, 25.37s/it]

 20%|████████████████▌                                                                  | 6/30 [02:32<10:16, 25.67s/it]

 23%|███████████████████▎                                                               | 7/30 [02:58<09:48, 25.58s/it]

 27%|██████████████████████▏  

Test Loss: 0.711122


Test Accuracy: 69% (416/600)


In [45]:
# load the model that got the best validation accuracy from previous training sessions
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

# train the model
n_epochs=15
model_transfer = train(n_epochs, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  1%|▊                                                                                 | 1/100 [00:24<40:04, 24.29s/it]

  2%|█▋                                                                                | 2/100 [00:51<41:09, 25.20s/it]

  3%|██▍                                                                               | 3/100 [01:18<41:47, 25.85s/it]

  4%|███▎                                                                              | 4/100 [01:45<41:37, 26.02s/it]

  5%|████                                                                              | 5/100 [02:11<41:08, 25.99s/it]

  6%|████▉                                                                             | 6/100 [02:40<42:02, 26.83s/it]

  7%|█████▋                                                                            | 7/100 [03:07<41:53, 27.03s/it]

  8%|██████▌                  

 67%|██████████████████████████████████████████████████████▎                          | 67/100 [27:49<13:31, 24.59s/it]

 68%|███████████████████████████████████████████████████████                          | 68/100 [28:12<12:55, 24.22s/it]

 69%|███████████████████████████████████████████████████████▉                         | 69/100 [28:38<12:47, 24.77s/it]

 70%|████████████████████████████████████████████████████████▋                        | 70/100 [29:01<12:02, 24.09s/it]

 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [29:26<11:47, 24.40s/it]

 72%|██████████████████████████████████████████████████████████▎                      | 72/100 [29:50<11:17, 24.21s/it]

 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [30:14<10:56, 24.30s/it]

 74%|███████████████████████████████████████████████████████████▉                     | 74/100 [30:38<10:32, 24.32s/it]

 75%|███████████████████████████

Epoch: 1 	Training Loss: 0.654738 	Validation Loss: 0.750791
Validation loss decreased (inf --> 0.750791).  Saving model ...




  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  1%|▊                                                                                 | 1/100 [00:24<40:16, 24.40s/it]

  2%|█▋                                                                                | 2/100 [00:49<40:22, 24.72s/it]

KeyboardInterrupt: 

In [50]:
# load the model that got the best validation accuracy from previous training sessions
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

optimizer_transfer = optim.Adagrad(model_transfer.classifier.parameters(), lr=0.01)
# train the model
n_epochs=15
model_transfer = train(n_epochs, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt', 0.750791)





  0%|                                                                                          | 0/100 [00:00<?, ?it/s]



  1%|▊                                                                                 | 1/100 [00:23<38:34, 23.38s/it]



  2%|█▋                                                                                | 2/100 [00:50<39:55, 24.45s/it]



  3%|██▍                                                                               | 3/100 [01:15<39:57, 24.71s/it]



  4%|███▎                                                                              | 4/100 [01:38<38:32, 24.09s/it]



  5%|████                                                                              | 5/100 [02:06<40:15, 25.42s/it]



  6%|████▉                                                                             | 6/100 [02:38<42:33, 27.17s/it]



  7%|█████▋                                                                            | 7/100 [03:10<44:35, 28.77s/it]



  8%|██████▌

 66%|█████████████████████████████████████████████████████▍                           | 66/100 [28:14<13:50, 24.44s/it]



 67%|██████████████████████████████████████████████████████▎                          | 67/100 [28:39<13:35, 24.71s/it]



 68%|███████████████████████████████████████████████████████                          | 68/100 [29:03<13:04, 24.52s/it]



 69%|███████████████████████████████████████████████████████▉                         | 69/100 [29:26<12:28, 24.13s/it]



 70%|████████████████████████████████████████████████████████▋                        | 70/100 [29:52<12:15, 24.53s/it]



 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [30:17<11:52, 24.57s/it]



 72%|██████████████████████████████████████████████████████████▎                      | 72/100 [30:40<11:22, 24.38s/it]



 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [31:04<10:54, 24.25s/it]



 74%|███████████

Epoch: 1 	Training Loss: 0.770108 	Validation Loss: 0.785357






  0%|                                                                                          | 0/100 [00:00<?, ?it/s]



  1%|▊                                                                                 | 1/100 [00:24<41:09, 24.95s/it]



  2%|█▋                                                                                | 2/100 [00:49<40:27, 24.77s/it]



  3%|██▍                                                                               | 3/100 [01:13<39:53, 24.68s/it]



  4%|███▎                                                                              | 4/100 [01:38<39:31, 24.71s/it]



  5%|████                                                                              | 5/100 [02:04<39:44, 25.10s/it]



  6%|████▉                                                                             | 6/100 [02:33<40:59, 26.16s/it]



  7%|█████▋                                                                            | 7/100 [02:57<39:41, 25.61s/it]



  8%|██████▌

 66%|█████████████████████████████████████████████████████▍                           | 66/100 [26:54<13:37, 24.05s/it]



 67%|██████████████████████████████████████████████████████▎                          | 67/100 [27:18<13:08, 23.90s/it]



 68%|███████████████████████████████████████████████████████                          | 68/100 [27:42<12:47, 23.97s/it]



 69%|███████████████████████████████████████████████████████▉                         | 69/100 [28:05<12:15, 23.74s/it]



 70%|████████████████████████████████████████████████████████▋                        | 70/100 [28:29<11:54, 23.81s/it]



 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [28:52<11:23, 23.58s/it]



 72%|██████████████████████████████████████████████████████████▎                      | 72/100 [29:19<11:29, 24.64s/it]



 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [29:44<11:09, 24.81s/it]



 74%|███████████

...





  0%|                                                                                          | 0/100 [00:00<?, ?it/s]



  1%|▊                                                                                 | 1/100 [00:23<38:50, 23.54s/it]



  2%|█▋                                                                                | 2/100 [00:47<38:37, 23.65s/it]



  3%|██▍                                                                               | 3/100 [01:12<38:47, 23.99s/it]



  4%|███▎                                                                              | 4/100 [01:37<39:12, 24.51s/it]



  5%|████                                                                              | 5/100 [02:00<37:59, 23.99s/it]



  6%|████▉                                                                             | 6/100 [02:24<37:38, 24.03s/it]



  7%|█████▋                                                                            | 7/100 [02:48<36:59, 23.86s/it]



  8%|██████▌

 66%|█████████████████████████████████████████████████████▍                           | 66/100 [28:45<16:48, 29.67s/it]



 67%|██████████████████████████████████████████████████████▎                          | 67/100 [29:18<16:53, 30.73s/it]



 68%|███████████████████████████████████████████████████████                          | 68/100 [29:51<16:46, 31.46s/it]



 69%|███████████████████████████████████████████████████████▉                         | 69/100 [30:22<16:10, 31.32s/it]



 70%|████████████████████████████████████████████████████████▋                        | 70/100 [30:54<15:45, 31.50s/it]



 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [31:25<15:07, 31.29s/it]



 72%|██████████████████████████████████████████████████████████▎                      | 72/100 [31:57<14:43, 31.55s/it]



 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [32:27<13:59, 31.09s/it]



 74%|███████████

Epoch: 6 	Training Loss: 0.616691 	Validation Loss: 0.782598






  0%|                                                                                          | 0/100 [00:00<?, ?it/s]



  1%|▊                                                                                 | 1/100 [00:25<41:50, 25.36s/it]



  2%|█▋                                                                                | 2/100 [00:51<41:37, 25.49s/it]



  3%|██▍                                                                               | 3/100 [01:19<42:28, 26.28s/it]



  4%|███▎                                                                              | 4/100 [01:43<41:03, 25.66s/it]



  5%|████                                                                              | 5/100 [02:10<41:21, 26.12s/it]



  6%|████▉                                                                             | 6/100 [02:35<40:19, 25.74s/it]



  7%|█████▋                                                                            | 7/100 [02:59<38:58, 25.14s/it]



  8%|██████▌

KeyboardInterrupt: 

In [48]:
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

In [49]:
test(loaders, model_transfer, criterion_transfer, use_cuda)





  0%|                                                                                           | 0/30 [00:00<?, ?it/s]



  3%|██▊                                                                                | 1/30 [00:25<12:26, 25.75s/it]



  7%|█████▌                                                                             | 2/30 [00:52<12:10, 26.09s/it]



 10%|████████▎                                                                          | 3/30 [01:16<11:26, 25.43s/it]



 13%|███████████                                                                        | 4/30 [01:42<11:03, 25.50s/it]



 17%|█████████████▊                                                                     | 5/30 [02:09<10:48, 25.94s/it]



 20%|████████████████▌                                                                  | 6/30 [02:35<10:25, 26.08s/it]



 23%|███████████████████▎                                                               | 7/30 [03:01<09:57, 25.96s/it]



 27%|███████

Test Loss: 0.672915


Test Accuracy: 72% (433/600)
